Maximum likelihood & Posterior distribution
===


Data: $\vec{x}$, $\vec{v}$

Model: $\Phi$, $\vec{\mu}_J$, $\vec{\Sigma}_J$

Likelihood: $P\left(\mathrm{data} \mid \mathrm{model}\right)$ = $P\left(\vec{x}, \vec{v} \mid \Phi, \vec{\mu}_J, \vec{\Sigma}_J\right)$ = $\mathcal{N}\left(J\left(\vec{x}, \vec{v}, \Phi \right); \vec{\mu}_J, \vec{\Sigma}_J\right)$

Free parameters:
--
Potential:
- Halo : a_NFWH, n_NFWH
- Disk : a_MND, b_MND, n_MND
- Bulge: a_HB, n_HB
- galpy: _REFV0_kms

= 8 free but put prioris on them (or keep a_MND, b_MND and a_HB fixed)

Distribution:
- $\vec{\mu} = \left(\mu_{J_R}, \mu_{J_z}, \mu_{L_z}\right)$ = 3 free
- $\vec{\Sigma}_J$ = symetric covariance matrix: 6 free


To do:
---
1. calculate maximum likelihood (e.g. use differential evolution for that)
2. calculate posterior distribution on model parameters (Bayes & emcee)

Bayes:
--
$P\left(A\mid B\right) = P\left(A\cap B\right)\div P\left(B\right)$

$P\left(B\mid A\right) = P\left(B\cap A\right)\div P\left(A\right)$


$P\left(A\cap B\right) = P\left(B\cap A\right)\Rightarrow P\left(A\mid B\right) = \frac{P\left(B\mid A\right)P\left(A\right)}{P\left(B\right)} = \frac{P\left(B\mid A\right)P\left(A\right)}{\int{P\left(B\mid A\right)P\left(A\right)\mathrm{d}modelparams}} $

Posterior = Likelihood $\cdot$ Prior / Normalization

Priors:
--
$n_{NFWH} + n_{MND} + n_{HB} = 1$

$\mu_{J_R} \geq0$

$\mu_{J_z} \geq0$

set a prior to minimize spread:
$P\left(\mathrm{det}\left(\Sigma\right)\right) = \frac{1}{\mathrm{det}\left(\Sigma\right)}$

if not strong enough try with $P\left(\mathrm{det}\left(\Sigma\right)\right) = \frac{1}{\mathrm{det}\left(\Sigma\right)^2}$
